# Computation Intelligence for Optimization | Sports League Optimization

`Group AM`
- Eduardo Mendes, 20240850
- Helena Duarte, 20240530
- João Freire, 20240528
- Mariana Sousa, 20240516

<div class="alert alert-block alert-info">

# Table of Contents
    
[1. Import Libraries](#1)<br>

[2. Load data](#2)<br>

<a class="anchor" id="1">

# 1. Import Libraries
    
</a>

In [1]:
import os
import pandas as pd

<a class="anchor" id="2">

# 2. Load data
    
</a>

In [2]:
DATA_DIR= os.path.join(os.getcwd(), 'players(in).csv')

df = pd.read_csv(DATA_DIR, index_col=0)

players = df.to_dict(orient="records")

df


,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95
5,Daniel Foster,DEF,90,110
6,Lucas Bennett,DEF,85,90
7,Owen Parker,DEF,88,100
8,Ethan Howard,DEF,80,70
9,Mason Reed,DEF,82,75


In [3]:
df["Name"].nunique() # eveyr name is different, so maybe we can use it as an ID

35

In [4]:
df["Position"].unique() 

array(['GK', 'DEF', 'MID', 'FWD'], dtype=object)

In [5]:
players

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95},
 {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
 {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
 {'Name': 'Mason Reed', 'Position': 'DEF', 'Skill': 82, 'Salary (€M)': 75},
 {'Name': 'Logan Brooks', 'Position': 'DEF', 'Skill': 86, 'Salary (€M)': 95},
 {'Name': 'Caleb Fisher', 'Position': 'DEF', 'Skill': 84, 'Salary (€M)': 85},
 {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Sa

In [8]:
df["Position"].value_counts()

Position
DEF    10
MID    10
FWD    10
GK      5
Name: count, dtype: int64

In [12]:
df["Skill"].describe()

count    35.000000
mean     86.400000
std       4.095909
min      79.000000
25%      83.500000
50%      86.000000
75%      89.500000
max      95.000000
Name: Skill, dtype: float64

In [10]:
df["Salary (€M)"].describe()

count     35.000000
mean      97.828571
std       19.876805
min       65.000000
25%       85.000000
50%       95.000000
75%      110.000000
max      150.000000
Name: Salary (€M), dtype: float64

<a class="anchor" id="3">

# 3. Problem Definiton
    
</a>

In a fantasy sports league, the objective is to assign players to teams in a way that ensures
a balanced distribution of talent while staying within salary caps.

1) Each player is defined by the following attributes:
* Skill rating: Represents the player's ability.
* Cost: The player's salary.
* Position (One of four roles) : Goalkeeper (GK), Defender (DEF), Midfielder (MID), or Forward (FWD).

A solution is a complete league configuration, specifying the team assignment for each player. These are the constraints that must be verified in every solution of the search space (no object is considered a solution if it doesn’t comply with these):
* Each team must consist of: 1 Goalkeeper, 2 Defenders, 2 Midfielders and 2
Forwards.
* Each player is assigned to exactly one team.

*Impossible Configurations*: Teams that do not follow this exact structure (e.g., a team with 2 goalkeepers, or a team where the same defender is assigned twice) are not part of the search space and are not considered solutions. It is forbidden to generate such an arrangement during evolution.

Besides that, each team should not exceed a 750€ million total budget. If it does, it is not a valid solution and the fitness value should reflect that.

The `objective` is to create a balanced league that complies with the constraints. 
A balanced league a is a league where the average skill rating of the players is roughly the same among the teams. 
This can be measured by the standard deviation of the average skill rating of the teams.

You can find a dataset of players with their names, position, skill rating and salary (in million €).
These players should be distributed across 5 teams of 7 players each.

In [5]:
def players_by_position(players):
    gk = []
    defenders = []
    midfielders = []
    forwards = []
    for i in players:
        if i["Position"] == "GK":
            gk.append(i)
        elif i["Position"] == "DEF":
            defenders.append(i)
        elif i["Position"] == "MID":
            midfielders.append(i)
        elif i["Position"] == "FWD":
            forwards.append(i)

    return gk, defenders, midfielders, forwards

In [6]:
gk, defenders, midfielders, forwards = players_by_position(players)
gk

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95}]

In [7]:
#vefify if team has the right number of players, and the right number of players for each position
def valid_team(team):
    n_gk = 0 #number of gk per team
    n_def = 0
    n_mid = 0
    n_forwards = 0

    for i in team:
        if i["Position"] == "GK":
            n_gk += 1
        elif i["Position"] == "DEF":
            n_def += 1
        elif i["Position"] == "MID":
            n_mid += 1
        elif i["Position"] == "FWD":
            n_forwards += 1

    if len(team) != 7:
        return False
    if n_gk != 1:
        return False
    if n_def != 2:
        return False
    if n_mid != 2:
        return False
    if n_forwards != 2:
        return False

    return True  

#function that sums all salaries of each player and returns the total
def total_cost(team):
    total_cost = 0

    for i in team:
        total_cost += + i["Salary (€M)"]
    return total_cost


#verify if each player is assigned to exactly one team
def has_repeated_players(solution):

    players = []
    for team in solution: #5 teams
        for player in team: #7 players for each team
            if player['Name'] in players:
                return True
            players.append(player['Name'])
    
    return False

In [8]:
#example of solutions to check 
solution1 = [
    [  # Team 1
        {'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
        {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Salary (€M)': 120},
        {'Name': 'Connor Hayes', 'Position': 'MID', 'Skill': 89, 'Salary (€M)': 105},
        {'Name': 'Sebastian Perry', 'Position': 'FWD', 'Skill': 95, 'Salary (€M)': 150},
        {'Name': 'Tyler Jenkins', 'Position': 'FWD', 'Skill': 80, 'Salary (€M)': 70}
        ],
    [  # Team 2
        {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
        {'Name': 'Dylan Morgan', 'Position': 'MID', 'Skill': 91, 'Salary (€M)': 115},
        {'Name': 'Hunter Cooper', 'Position': 'MID', 'Skill': 83, 'Salary (€M)': 85},
        {'Name': 'Xavier Bryant', 'Position': 'FWD', 'Skill': 90, 'Salary (€M)': 120},
        {'Name': 'Adrian Collins', 'Position': 'FWD', 'Skill': 85, 'Salary (€M)': 90}
    ]
]

#example of solution with repeated name
solution2 = [
    [  # Team 1
        {'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90}, #repeated
        {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Salary (€M)': 120},
        {'Name': 'Connor Hayes', 'Position': 'MID', 'Skill': 89, 'Salary (€M)': 105},
        {'Name': 'Sebastian Perry', 'Position': 'FWD', 'Skill': 95, 'Salary (€M)': 150},
        {'Name': 'Tyler Jenkins', 'Position': 'FWD', 'Skill': 80, 'Salary (€M)': 70}
    ],
    [  # Team 2
        {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
        {'Name': 'Dylan Morgan', 'Position': 'MID', 'Skill': 91, 'Salary (€M)': 115},
        {'Name': 'Hunter Cooper', 'Position': 'MID', 'Skill': 83, 'Salary (€M)': 85},
        {'Name': 'Xavier Bryant', 'Position': 'FWD', 'Skill': 90, 'Salary (€M)': 120},
        {'Name': 'Adrian Collins', 'Position': 'FWD', 'Skill': 85, 'Salary (€M)': 90}
    ]
]

#example of solution with 6 players
solution3 = [
    [  # Team 1
        {'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
        {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Salary (€M)': 120},
        {'Name': 'Connor Hayes', 'Position': 'MID', 'Skill': 89, 'Salary (€M)': 105},
        {'Name': 'Sebastian Perry', 'Position': 'FWD', 'Skill': 95, 'Salary (€M)': 150},
        {'Name': 'Tyler Jenkins', 'Position': 'FWD', 'Skill': 80, 'Salary (€M)': 70}
    ],
    [  # Team 2
        {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
        {'Name': 'Dylan Morgan', 'Position': 'MID', 'Skill': 91, 'Salary (€M)': 115},
        {'Name': 'Hunter Cooper', 'Position': 'MID', 'Skill': 83, 'Salary (€M)': 85},
        {'Name': 'Xavier Bryant', 'Position': 'FWD', 'Skill': 90, 'Salary (€M)': 120}
        ]
]

In [9]:

print("has repeated players:")
print("Solution 1:", has_repeated_players(solution1))
print("Solution 2:", has_repeated_players(solution2))
print("Solution 3:", has_repeated_players(solution3))

print("\nTotal cost solution 1:")
print("Team 1:", total_cost(solution1[0]), "M €")
print("Team 2:", total_cost(solution1[1]), "M €")

print("\nSolution 2 has valid teams:")
print("Team 1:", valid_team(solution2[0]))
print("Team 2:", valid_team(solution2[1]))

has repeated players:
Solution 1: False
Solution 2: True
Solution 3: False

Total cost solution 1:
Team 1: 735 M €
Team 2: 680 M €

Solution 2 has valid teams:
Team 1: False
Team 2: True


In [13]:
df.columns

Index(['Name', 'Position', 'Skill', 'Salary (€M)'], dtype='object')

In [14]:
class Player:
    def __init__(self,name, position, skill, salary):
        self.name = name
        self.position = position
        self.skill = skill
        self.salary = salary


# Selection Operators

# Crossover Operators

# Mutation Operators